In [1]:
import time
import random
import requests
import pandas as pd
import datetime
from tqdm import tqdm
import urllib.parse
import os
from fake_useragent import UserAgent
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Configuration
DEFAULT_START_DATE = "2010-01-01"
DEFAULT_END_DATE = datetime.date.today().strftime("%Y-%m-%d")
AREA_CODE_FILE = "Centanet_ICI_Area_Code.xlsx"
BASE_URL = "https://oir.centanet.com/api/Transaction/GetTransactionList"
PAGESIZE = 10000

def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def get_cookies():
    # This is a placeholder. In a real scenario, you'd implement a way to get fresh cookies.
    return {
        "cookie1": f"value1_{random.randint(1000, 9999)}",
        "cookie2": f"value2_{random.randint(1000, 9999)}"
    }

def create_session():
    session = requests.Session()
    retry = Retry(total=3, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def main():
    try:
        start_dt = datetime.datetime.strptime(DEFAULT_START_DATE, "%Y-%m-%d")
        end_dt = datetime.datetime.strptime(DEFAULT_END_DATE, "%Y-%m-%d")
    except Exception as e:
        print("Error parsing dates:", e)
        return

    start_api = start_dt.strftime("%d/%m/%Y")
    end_api = end_dt.strftime("%d/%m/%Y")
    date_range = f"{start_api}-{end_api}"
    date_range_encoded = urllib.parse.quote(date_range)

    try:
        area_df = pd.read_excel(AREA_CODE_FILE, engine="openpyxl")
    except Exception as e:
        print(f"Error reading {AREA_CODE_FILE}:", e)
        return

    output_file = f"{datetime.date.today().strftime('%Y-%m-%d')}_centanet_ici_transaction.csv"
    if os.path.exists(output_file):
        os.remove(output_file)

    session = create_session()
    cookies = get_cookies()

    for idx, row in tqdm(area_df.iterrows(), total=area_df.shape[0], desc="Processing Areas"):
        if idx % 10 == 0:
            cookies = get_cookies()
            session = create_session()

        region = row["Region"]
        district = row["District"]
        code = row["Code"]

        #print(f"\nProcessing area: {district} (Code: {code}, Region: {region})")
        page_index = 1
        area_results = []

        while True:
            url = (f"{BASE_URL}?pageindex={page_index}&pagesize={PAGESIZE}"
                   f"&daterang={date_range_encoded}&posttype=B&districtids={code}&lang=EN")
            
            headers = {
                "User-Agent": get_random_user_agent(),
                "Accept": "application/json, text/plain, */*",
                "Accept-Language": "en-US,en;q=0.9",
                "Referer": "https://oir.centanet.com/",
                "Origin": "https://oir.centanet.com",
                "Connection": "keep-alive"
            }

            #print(f"Requesting page {page_index} for area {district} …")
            try:
                response = session.get(url, headers=headers, cookies=cookies, timeout=20)
                response.raise_for_status()
                json_data = response.json()
            except requests.exceptions.RequestException as e:
                print("Error during the API request for area", district, ":", e)
                break

            if json_data.get("responseCode") != 1:
                print("API response not successful; ending pagination for", district)
                break

            items = json_data.get("data", {}).get("recordList", {}).get("items", [])
            if not items:
                #print(f"No items found on page {page_index} for area {district}.")
                break

            for item in items:
                item["Region"] = region
                item["District"] = district
                item["AreaCode"] = code
                area_results.append(item)
            
            #print(f"Fetched {len(items)} items on page {page_index} for area {district}.")
            page_index += 1

            time.sleep(random.uniform(3, 5))

        if area_results:
            df_area = pd.DataFrame(area_results)
            df_area.to_csv(output_file, mode="a", index=False, header=not os.path.exists(output_file), encoding="utf-8-sig")
            #print(f"Saved {len(area_results)} items for area {district} into {output_file}.")
        else:
            print(f"No data collected for area {district}.")
        
        time.sleep(random.uniform(5, 7))

    print("\nScraping complete. All data saved in:", output_file)

if __name__ == "__main__":
    main()


Processing Areas: 100%|██████████| 53/53 [17:50<00:00, 20.20s/it]


Scraping complete. All data saved in: 2025-04-13_centanet_ici_transaction.csv


In [17]:
#simple cleansing
import pandas as pd 

df_clean = pd.read_csv('2025-02-27_centanet_ici_transaction.csv')
print(df_clean.shape)

/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_84647/2354615037.py:4: DtypeWarning: Columns (6,16,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_clean = pd.read_csv('2025-02-27_centanet_ici_transaction.csv')


(348777, 34)


In [ ]:
import ast

df_cleansed = df_clean[['id', 'deptDisplayName', 'centabldg', 'transactionDate', 'transactionType',
                        'propertyNameCn', 'propertyNameEn', 'propertyUsageDisplayName','floor',
                        'unit', 'isPriceEstimated', 'transactionArea', 'sourceDisplayName',
                        'priceInfo', 'ibsContractID', 'addressDisplayName', 'Region', 'District',
                        'AreaCode']]

def safe_get_value(x, key, default=None):
    if isinstance(x, str):
        try:
            x = ast.literal_eval(x)
        except (SyntaxError, ValueError):
            return default
    if isinstance(x, dict):
        return x.get(key, default)
    return default

# Use .loc columns assignment on the DataFrame copy
df_cleansed.loc[:, 'price'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'price'))
# df_cleansed.loc[:, 'priceDisplayName'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'priceDisplayName'))
df_cleansed.loc[:, 'pricePostTypeDisplayName'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'pricePostTypeDisplayName'))
df_cleansed.loc[:, 'avgPrice'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'avgPrice'))
#df_cleansed.loc[:, 'avgPriceDisplayName'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'avgPriceDisplayName'))
df_cleansed.loc[:, 'rental'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'rental'))
# df_cleansed.loc[:, 'rentalDisplayName'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'rentalDisplayName'))
df_cleansed.loc[:, 'rentPostTypeDisplayName'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'rentPostTypeDisplayName'))
df_cleansed.loc[:, 'avgRental'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'avgRental'))
#df_cleansed.loc[:, 'avgRentalDisplayName'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'avgRentalDisplayName'))
#df_cleansed.loc[:, 'gains_Price'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'gains_Price'))
#df_cleansed.loc[:, 'gains_Rental'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'gains_Rental'))
#df_cleansed.loc[:, 'priceTo'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'priceTo'))
#df_cleansed.loc[:, 'rentalTo'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'rentalTo'))
#df_cleansed.loc[:, 'unitPriceTo'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'unitPriceTo'))
#df_cleansed.loc[:, 'unitRentalTo'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'unitRentalTo'))
#df_cleansed.loc[:, 'priceDesc'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'priceDesc'))
#df_cleansed.loc[:, 'fhPriceInfo'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'fhPriceInfo'))
#df_cleansed.loc[:, 'fhAvgPriceInfo'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'fhAvgPriceInfo'))
#df_cleansed.loc[:, 'fhRentInfo'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'fhRentInfo'))
#df_cleansed.loc[:, 'fhAvgRentInfo'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'fhAvgRentInfo'))

# Optionally, drop the original priceInfo containing the dictionaries
df_cleansed.drop(columns=['priceInfo'], inplace=True)

df_cleansed.to_excel('2025-02-27_centanet_ici_transaction.xlsx', index=False)

print(f'After Selecting Columns, the updated shape is: {df_cleansed.shape}.')
	
					
			
#436.5MB csv -> 53.1MB xlsx -> 51MB xlsx

/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_84647/3232419267.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleansed.loc[:, 'price'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'price'))
/var/folders/rx/s42f27kn5j5ddqzlhz7qr2yw0000gn/T/ipykernel_84647/3232419267.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleansed.loc[:, 'pricePostTypeDisplayName'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'pricePostTypeDisplayName'))
/var/folders/rx/s4

After Selecting Columns, the updated shape is: (348777, 24).


In [1]:
#gpt updated code - in one piece

In [1]:
import time
import random
import requests
import pandas as pd
import datetime
from tqdm import tqdm
import urllib.parse
import os
import ast
from fake_useragent import UserAgent
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Configuration
DEFAULT_START_DATE = "2010-01-01"
DEFAULT_END_DATE = datetime.date.today().strftime("%Y-%m-%d")
AREA_CODE_FILE = "Centanet_ICI_Area_Code.xlsx"
BASE_URL = "https://oir.centanet.com/api/Transaction/GetTransactionList"
PAGESIZE = 10000

def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def get_cookies():
    return {
        "cookie1": f"value1_{random.randint(1000, 9999)}",
        "cookie2": f"value2_{random.randint(1000, 9999)}"
    }

def create_session():
    session = requests.Session()
    retry = Retry(total=3, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def safe_get_value(x, key, default=None):
    if isinstance(x, str):
        try:
            x = ast.literal_eval(x)
        except (SyntaxError, ValueError):
            return default
    if isinstance(x, dict):
        return x.get(key, default)
    return default

def scrape_and_clean_centanet_data(start_date=DEFAULT_START_DATE, end_date=DEFAULT_END_DATE, area_code_file=AREA_CODE_FILE):
    try:
        start_dt = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    except Exception as e:
        print("Error parsing dates:", e)
        return

    start_api = start_dt.strftime("%d/%m/%Y")
    end_api = end_dt.strftime("%d/%m/%Y")
    date_range = f"{start_api}-{end_api}"
    date_range_encoded = urllib.parse.quote(date_range)

    try:
        area_df = pd.read_excel(area_code_file, engine="openpyxl")
    except Exception as e:
        print(f"Error reading {area_code_file}:", e)
        return

    output_file = f"{datetime.date.today().strftime('%Y-%m-%d')}_centanet_ici_transaction.csv"
    if os.path.exists(output_file):
        os.remove(output_file)

    session = create_session()
    cookies = get_cookies()

    for idx, row in tqdm(area_df.iterrows(), total=area_df.shape[0], desc="Processing Areas"):
        if idx % 10 == 0:
            cookies = get_cookies()
            session = create_session()

        region = row["Region"]
        district = row["District"]
        code = row["Code"]

        page_index = 1
        area_results = []

        while True:
            url = (f"{BASE_URL}?pageindex={page_index}&pagesize={PAGESIZE}"
                   f"&daterang={date_range_encoded}&posttype=B&districtids={code}&lang=EN")
            
            headers = {
                "User-Agent": get_random_user_agent(),
                "Accept": "application/json, text/plain, */*",
                "Accept-Language": "en-US,en;q=0.9",
                "Referer": "https://oir.centanet.com/",
                "Origin": "https://oir.centanet.com",
                "Connection": "keep-alive"
            }

            try:
                response = session.get(url, headers=headers, cookies=cookies, timeout=20)
                response.raise_for_status()
                json_data = response.json()
            except requests.exceptions.RequestException as e:
                print("Error during the API request for area", district, ":", e)
                break

            if json_data.get("responseCode") != 1:
                print("API response not successful; ending pagination for", district)
                break

            items = json_data.get("data", {}).get("recordList", {}).get("items", [])
            if not items:
                break

            for item in items:
                item["Region"] = region
                item["District"] = district
                item["AreaCode"] = code
                area_results.append(item)
            
            page_index += 1
            time.sleep(random.uniform(3, 5))

        if area_results:
            df_area = pd.DataFrame(area_results)
            df_area.to_csv(output_file, mode="a", index=False, header=not os.path.exists(output_file), encoding="utf-8-sig")
        else:
            print(f"No data collected for area {district}.")
        
        time.sleep(random.uniform(5, 7))

    print("\nScraping complete. All data saved in:", output_file)

    # Data cleaning
    df_clean = pd.read_csv(output_file)
    
    df_cleansed = df_clean[['id', 'deptDisplayName', 'centabldg', 'transactionDate', 'transactionType',
                            'propertyNameCn', 'propertyNameEn', 'propertyUsageDisplayName','floor',
                            'unit', 'isPriceEstimated', 'transactionArea', 'sourceDisplayName',
                            'priceInfo', 'ibsContractID', 'addressDisplayName', 'Region', 'District',
                            'AreaCode']]

    df_cleansed.loc[:, 'price'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'price'))
    df_cleansed.loc[:, 'pricePostTypeDisplayName'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'pricePostTypeDisplayName'))
    df_cleansed.loc[:, 'avgPrice'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'avgPrice'))
    df_cleansed.loc[:, 'rental'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'rental'))
    df_cleansed.loc[:, 'rentPostTypeDisplayName'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'rentPostTypeDisplayName'))
    df_cleansed.loc[:, 'avgRental'] = df_cleansed['priceInfo'].apply(lambda x: safe_get_value(x, 'avgRental'))

    df_cleansed.drop(columns=['priceInfo'], inplace=True)

    cleaned_output_file = f"{datetime.date.today().strftime('%Y-%m-%d')}_centanet_ici_transaction_cleaned.csv"
    df_cleansed.to_csv(cleaned_output_file, index=False, encoding="utf-8-sig")

    print(f'Data cleaning complete. Cleaned data saved in: {cleaned_output_file}')
    print(f'Final dataframe shape: {df_cleansed.shape}')

if __name__ == "__main__":
    scrape_and_clean_centanet_data()


Processing Areas:   0%|          | 0/53 [00:04<?, ?it/s]


KeyboardInterrupt: 